In [ ]:
import pandas as pd
import requests
import os
import random
import time
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from simple_salesforce import Salesforce
from dotenv import load_dotenv
from faker import Faker

load_dotenv()

# HAPI FHIR

In [ ]:
BASE_URL = 'https://hapi.fhir.org/baseR4'
organization_id = os.getenv('HAPI_ORG_ID')

## Organization: Data types creation

### Main record types

In [ ]:
# # Organization

# new_org = {
#     'resourceType': 'Organization',
#     'name': 'JSC Health Integration Demo',
#     'alias': ['FHIR Portfolio Project'],
#     'telecom': [
#         {'system': 'phone', 'value': '+1-555-000-0000'},
#         {'system': 'email', 'value': 'info@demo-health.org.invalid'}
#     ],
#     'address': [{
#         'line': ['123 Demo Street'],
#         'city': 'Miami',
#         'state': 'FL',
#         'postalCode': '33186',
#         'country': 'US'
#     }],
#     'active': True
# }

# response = requests.post(
#     f'{BASE_URL}/Organization',
#     headers={'Content-Type': 'application/fhir+json'},
#     json=new_org
# )

# print(response.status_code, response.json())


In [ ]:
# # Locations

# new_location = {
#     'resourceType': 'Location',
#     'name': 'JSC South Miami Clinic',
#     'status': 'active',
#     'telecom': [{'system': 'phone', 'value': '305-111-2233'}],
#     'address': {
#         'line': ['12345 Sunset Drive'],
#         'city': 'Miami',
#         'state': 'FL',
#         'postalCode': '33156'
#     },
#     'managingOrganization': {
#         'reference': 'Organization/49384657'  # <-- New Org Id
#     }
# }

# response = requests.post(
#     f'{BASE_URL}/Location',
#     headers={'Content-Type': 'application/fhir+json'},
#     json=new_location
# )

# print(response.status_code, response.json())

In [ ]:
# # Practitioners

# new_practitioner = {
#     'resourceType': 'Practitioner',
#     'name': [
#         {
#             "use": "official",
#             'family': "Testing",
#             "given": ["Sebastian"]
#         }
#     ],
#     "telecom": [
#         {"system": "phone", "value": "305-123-4567", "use": "work"},
#         {"system": "email", "value": "stest@demo-health.org.invalid", "use": "work"}
#     ],
#     "address": [
#         {
#             "line": ["123 Clinic St"],
#             "city": "Miami",
#             "state": "FL",
#             "postalCode": "33193"
#         }
#     ],
#     "gender": "male",
#     'active': True
# }

# response = requests.post(f'{BASE_URL}/Practitioner', json=new_practitioner)

# print(response.status_code, response.json())

In [ ]:
# # Practitioner Role

# practitioner_id = '49405319'       # from Step 1
# organization_id = '49384657'       # the Organization you created earlier
# location_id = '49385197'            # one of your Locations (49385010, 49385115, 49385197)

# role = {
#     'resourceType': 'PractitionerRole',
#     'practitioner': {
#         'reference': f'Practitioner/{practitioner_id}'
#     },
#     'organization': {
#         'reference': f'Organization/{organization_id}'
#     },
#     'location': [
#         {'reference': f'Location/{location_id}'}
#     ],
#     'specialty': [
#         {
#             'coding': [
#                 {
#                     'system': 'https://taxonomy.nucc.org/',
#                     'code': '208D00000X',
#                     'display': 'General Practice Physician'
#                 }
#             ],
#             'text': 'General Practice Physician'
#         }
#     ],
#     # 'specialty': [
#     #     {
#     #         'coding': [
#     #             {
#     #                 'system': 'https://taxonomy.nucc.org/',
#     #                 'code': '363LF0000X',
#     #                 'display': 'Family Nurse Practitioner'
#     #             }
#     #         ],
#     #         'text': 'Family Nurse Practitioner'
#     #     }
#     # ],
#     'active': True
# }

# response = requests.post(f'{BASE_URL}/PractitionerRole', json=role)
# print(response.status_code, response.json())

### Patients mock data

In [ ]:
# fake = Faker()

# AREA_CODES = ['305','786']
# EMAIL_DOMAINS = ['@gmail.com.invalid','@outlook.com.invalid','@aol.com.invalid','@yahoo.com.invalid']
# CITIES = ['Miami', 'Hialeah', 'Coral Gables', 'Miami Beach']
# ZIPCODES = ['33127','33137', '33140', '33130', '33150', '33156', '33126', '33183', '33186', '33193']

# def generate_patient_data(n: int, organization_id: str) -> pd.DataFrame:
#     """"
#     Generate mock patient data for Miami-Dade area.
#     """
#     patients = []
#     for _ in range(n):
#         gender = random.choice(['male', 'female'])
#         first_name = fake.first_name_male() if gender == 'male' else fake.first_name_female()
#         last_name = fake.last_name()
#         birth_date = fake.date_of_birth(minimum_age=18, maximum_age=90).isoformat()

#         phone = random.choice(AREA_CODES)+ f'-{random.randint(200,999)}-{random.randint(1000,9999)}'
#         email = f'{first_name.lower()}.{last_name.lower()}' + random.choice(EMAIL_DOMAINS)

#         address = {
#             'line': [fake.street_address()],
#             'city': random.choice(CITIES),
#             'state': 'FL',
#             'postalCode': random.choice(ZIPCODES)
#         }

#         patients.append({
#             'first_name': first_name,
#             'last_name': last_name,
#             'birthDate': birth_date,
#             'gender': gender,
#             'phone': phone,
#             'email': email,
#             'address': address,
#             'organization_id': organization_id
#         })

#     return pd.DataFrame(patients)

In [ ]:
# patients_df = generate_patient_data(100, organization_id)
# patients_df

In [ ]:
# # Load patients into HAPI FHIR

# def post_patient(row) -> dict:
#     """
#     POST a Patient resource to HAPI FHIR.
#     """
#     patient = {
#         'resourceType': 'Patient',
#         'name': [{'use': 'official', 'family': row['last_name'], 'given': [row['first_name']]}],
#         'gender': row['gender'],
#         'birthDate': row['birthDate'],
#         'telecom': [
#             {'system': 'phone', 'value': row['phone'], 'use': 'mobile'},
#             {'system': 'email', 'value': row['email'], 'use': 'home'}
#         ],
#         'address': [row['address']],
#         'managingOrganization': {'reference': f"Organization/{row['organization_id']}"}
#     }

#     response = requests.post(f'{BASE_URL}/Patient', json=patient)
#     response.raise_for_status()
#     return response.json()

In [ ]:
# for _, row in patients_df.iterrows():
#     response = post_patient(row)
#     print("Created Patient:", response.get("id"))

### Appointments mock data

In [ ]:
# # Random Appointment generator function

# TODAY = datetime(2025, 9, 26)
# PAST_STATUSES = ['fulfilled', 'cancelled']
# FUTURE_STATUSES = ['booked', 'cancelled']

# def random_weekday(start_date, end_date):
#     """Pick a random weekday datetime between start_date and end_date."""
#     while True:
#         dt = start_date + timedelta(
#             days=random.randint(0, (end_date - start_date).days),
#             hours=random.randint(8, 16),
#             minutes=random.choice([0, 15, 30, 45])
#         )
#         if dt.weekday() < 5:  # Mon–Fri only
#             return dt

# def generate_appointments(patients_df, providers_df, locations_df, work_types, n=500):
#     """
#     Generate random appointments DataFrame using available patients, providers, locations, work_types.
#     """
#     start_date = datetime(2025, 1, 1)
#     end_date = datetime(2026, 6, 30)

#     records = []
#     for _ in range(n):
#         patient = random.choice(patients_df["patient_id"].tolist())
#         provider = random.choice(providers_df["practitioner_id"].tolist())
#         location = random.choice(locations_df["location_id"].tolist())
#         work_key, work = random.choice(list(work_types.items()))

#         start_dt = random_weekday(start_date, end_date)
#         end_dt = start_dt + timedelta(minutes=random.choice([15, 30, 45, 60]))

#         if start_dt < TODAY:
#             status = random.choice(PAST_STATUSES)
#         else:
#             status = random.choice(FUTURE_STATUSES)

#         records.append({
#             "patient_id": patient,
#             "provider_id": provider,
#             "location_id": location,
#             "work_type": work["display"],
#             "start": start_dt,
#             "end": end_dt,
#             "status": status
#         })

#     return pd.DataFrame(records)

In [ ]:
# # POST request: Load Appointment records
# def post_appointments(appointments_df, work_types, base_url=BASE_URL, sleep_time=0.2):
#     """
#     POST Appointment records to HAPI FHIR server.
    
#     Args:
#         appointments_df (pd.DataFrame): DataFrame with columns
#             [patient_id, provider_id, location_id, work_type, start, end, status]
#         work_types (dict): dictionary with work types, keyed by display or code
#         base_url (str): HAPI FHIR server base URL
#         sleep_time (float): delay between POST requests to avoid throttling
    
#     Returns:
#         list of dict: server responses
#     """
#     responses = []
    
#     total = len(appointments_df)
#     print(f"Starting to POST {total} appointments...")

#     for i, row in appointments_df.iterrows():
#         # find the work type dict (lookup by display value)
#         wk = next((v for v in work_types.values() if v["display"] == row["work_type"]), None)
        
#         appt = {
#             "resourceType": "Appointment",
#             "status": row["status"],
#             "serviceType": [
#                 {
#                     "coding": [
#                         {
#                             "system": wk["system"],
#                             "code": wk["code"],
#                             "display": wk["display"]
#                         }
#                     ],
#                     "text": wk["display"]
#                 }
#             ],
#             "start": row["start"].strftime("%Y-%m-%dT%H:%M:%SZ"),
#             "end": row["end"].strftime("%Y-%m-%dT%H:%M:%SZ"),
#             "participant": [
#                 {"actor": {"reference": f"Patient/{row['patient_id']}"}, "status": "accepted"},
#                 {"actor": {"reference": f"Practitioner/{row['provider_id']}"}, "status": "accepted"},
#                 {"actor": {"reference": f"Location/{row['location_id']}"}, "status": "accepted"},
#             ]
#         }

#         resp = requests.post(
#             f"{base_url}/Appointment",
#             headers={"Content-Type": "application/fhir+json"},
#             json=appt
#         )

#         # store response info
#         try:
#             resp.raise_for_status()
#             fhir_id = resp.json().get("id")
#             responses.append({"id": fhir_id, "status": resp.status_code})
#             print(f"[{i+1}/{total}] ✅ Created Appointment (ID: {fhir_id})")
#         except Exception:
#             responses.append({"error": resp.text, "status": resp.status_code})
#             print(f"[{i+1}/{total}] ❌ Error {resp.status_code}")

#         time.sleep(sleep_time)
    
#     print("Finished posting appointments.")
#     return responses

In [ ]:
# # Appointments data load
# """
# This section must run once all other record times are loaded and retrieved from the EHR. It uses the resulting dataframes to get the random data.
# """

# appointments_df = generate_appointments(patients_df, providers_df, location_df, WORK_TYPES, n=500)
# appointments_df.head()

In [ ]:
# responses = post_appointments(appointments_df, WORK_TYPES, base_url=BASE_URL, sleep_time=1)

## Data Collection

In [ ]:
def fetch_resource(resource_type: str, search_criteria: str) -> dict:
    """
    Fetch a resource from HAPI FHIR public server.

    Args:
        resource_type (str): e.g. 'Patient', 'Practitioner', 'Appointment'
        search_criteria (str): Search criteria field and its value
    
    Returns:
        dict: JSON response
    """
    url = f"{BASE_URL}/{resource_type}/_search?{search_criteria}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def fetch_all_resources(url: str, params: dict = None) -> list:
    """
    Fetch all resources from a paginated HAPI FHIR endpoint.

    Args:
        url (str): Base resource URL (e.g., f"{BASE_URL}/Appointment").
        params (dict): Query params (e.g., {'actor': 'Practitioner/123'}).

    Returns:
        list: Combined list of all resource entries across pages.
    """
    entries = []
    while url:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        entries.extend(data.get("entry", []))
        
        # look for "next" link in bundle
        url = None
        for link in data.get("link", []):
            if link.get("relation") == "next":
                url = link.get("url")
                params = None
                break

    return entries

In [ ]:
#Locations:
def locations_to_df(response: dict) -> pd.DataFrame:
    """
    Normalize FHIR Location resources into a DataFrame.
    """
    locations = []
    for entry in response.get('entry', []):
        response = entry['resource']
        
        location_id = response.get('id')
        name = response.get('name')
        source = response.get('name')

        # Telecom (grab first phone if available)
        phone = None
        if 'telecom' in response:
            for t in response['telecom']:
                if t.get('system') == 'phone':
                    phone = t.get('value')
                    break
        
        status =  response.get('status')

        # Address:
        address = response.get('address', {})
        address_line = ' '.join(address.get('line', [])) if address else None
        city = address.get('city')
        state = address.get('state')
        zip_code = address.get('postalCode')

        locations.append({
            'location_id': location_id,
            'name': name,
            'phone': phone,
            'status': status,
            'address_line': address_line,
            'city': city,
            'state': state,
            'zip_code': zip_code
        })

    return pd.DataFrame(locations)

In [ ]:
# Practitioners:
def get_practitioner_roles(org_id: str, count: int = 50) -> dict:
    """
    Fetch PractitionerRole resources for a given Organization.
    """
    url = f'{BASE_URL}/PractitionerRole'
    params = {
        '_count': count,
        'organization': f'Organization/{org_id}'
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    return resp.json()

def get_practitioner(practitioner_id: str) -> dict:
    """
    Fetch a single Practitioner resource by ID.
    """
    url = f'{BASE_URL}/Practitioner/{practitioner_id}'
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json()

def providers_to_df(org_id: str, count: int = 50) -> pd.DataFrame:
    """
    Build dataframe with provider details (Practitioner + Role).
    """
    roles = get_practitioner_roles(org_id, count)
    providers = []

    for entry in roles.get('entry', []):
        role = entry['resource']

        practitioner_ref = role.get('practitioner', {}).get('reference')
        if not practitioner_ref:
            continue

        practitioner_id = practitioner_ref.split('/')[-1]
        practitioner = get_practitioner(practitioner_id)

        # ---- Practitioner data ----
        name_data = practitioner.get('name', [{}])[0]
        family = name_data.get('family')
        given = ' '.join(name_data.get('given', []))

        telecom = practitioner.get('telecom', [])
        phone = next((t['value'] for t in telecom if t.get('system') == 'phone'), None)
        email = next((t['value'] for t in telecom if t.get('system') == 'email'), None)

        # ---- PractitionerRole data ----
        specialty = None
        if 'specialty' in role:
            coding = role['specialty'][0].get('coding', [])
            if coding:
                specialty = coding[0].get('display')
            else:
                specialty = role['specialty'][0].get('text')

        locations = [loc['reference'].split('/')[-1] for loc in role.get('location', [])]

        providers.append({
            'practitioner_id': practitioner_id,
            'first_name': given,
            'last_name': family,
            'phone': phone,
            'email': email,
            'specialty': specialty,
            'location_id': ', '.join(locations)
        })

    return pd.DataFrame(providers)

In [ ]:
# Patients:
def patients_to_df(response: dict) -> pd.DataFrame:
    """
    Normalize FHIR Patient resources into a DataFrame.
    
    Parameters
    ----------
    response : dict
        Bundle response from FHIR Patient search or read operation.
    
    Returns
    -------
    pd.DataFrame
        Flattened patient data with key demographic and contact fields.
    """
    patients = []
    for entry in response.get('entry', []):
        res = entry.get('resource', {})
        
        patient_id = res.get('id')

        # Name (take first if multiple)
        name = None
        if 'name' in res and res['name']:
            given = ' '.join(res['name'][0].get('given', []))
            family = res['name'][0].get('family', '')
            name = f"{given} {family}".strip()

        # Birthdate & gender
        birth_date = res.get('birthDate')
        gender = res.get('gender')

        # Telecom (phone & email if available)
        phone, email = None, None
        for t in res.get('telecom', []):
            if t.get('system') == 'phone' and not phone:
                phone = t.get('value')
            elif t.get('system') == 'email' and not email:
                email = t.get('value')

        # Address (take first if multiple)
        address = res.get('address', [{}])[0]
        address_line = ' '.join(address.get('line', []))
        city = address.get('city')
        state = address.get('state')
        postal = address.get('postalCode')

        # Organization (reference only)
        org_ref = res.get('managingOrganization', {}).get('reference')

        patients.append({
            'patient_id': patient_id,
            'name': name,
            'birth_date': birth_date,
            'gender': gender,
            'phone': phone,
            'email': email,
            'address_line': address_line,
            'city': city,
            'state': state,
            'postal_code': postal,
            'organization': org_ref
        })

    return pd.DataFrame(patients)

In [ ]:
locations_df = locations_to_df(fetch_resource('Location', f'organization={organization_id}'))

In [ ]:
dict_rename = {
    'name': 'Name',
    'location_id': 'EHR_Location_Id__c',
    'address_line': 'Street',
    'city': 'City',
    'state': 'State',
    'zip_code': 'PostalCode'
}
locations_df = (locations_df.drop(columns=['phone', 'status']).rename(columns=(dict_rename)))
locations_df['OperatingHoursId'] = '0OHbm000002EyftGAC'

In [ ]:
providers_df = providers_to_df(organization_id, count=10)

In [ ]:
patients_df = patients_to_df(fetch_resource('Patient', f'organization={organization_id}'))

In [ ]:
patients_df[["First_Name__c", "Last_Name__c"]] = patients_df["name"].str.split(" ", n=1, expand=True)
dict_rename = {
    'birth_date': 'Date_of_Birth__c',
    'patient_id': 'EHR_Patient_Id__c',
    'address_line': 'Address_Line__c',
    'city': 'City__c',
    'state': 'State__c',
    'postal_code': 'Postal_Code__c',
    'phone': 'Phone',
    'email': 'Email__c'
}
patients_df = (patients_df.drop(columns=['name', 'organization','gender']).rename(columns=(dict_rename)))

In [ ]:
users_df = providers_df[['first_name', 'last_name', 'email']].copy()
dict_rename = {
    'first_name': 'FirstName',
    'last_name': 'LastName',
    'email': 'Email'
}
users_df = users_df.rename(columns=(dict_rename))
users_df['ProfileId'] = '00ebm00000Dvu6VAAR'
# users_df['UserRoleId'] = '00Ebm00000CpdqAEAR'
users_df['Alias'] = (users_df['FirstName'].str[0] + users_df['LastName'].str[:4]).str.lower().str.ljust(5, 'x')
users_df['CommunityNickname'] = users_df['Alias']
users_df['IsActive'] = True
users_df['Username'] = users_df['Email']
users_df['TimeZoneSidKey'] = 'America/New_York'
users_df['EmailEncodingKey'] = 'UTF-8'
users_df['LanguageLocaleKey'] = 'en_US'
users_df['LocaleSidKey'] = 'en_US'

In [ ]:
# Work Types dictionary:
""""
HAPI FHIR does not provide a dedicated endpoint to create reusable "work type" records.
Instead, the Appointment resource includes a `serviceType` field (CodeableConcept) 
where work types can be represented. 

For this project, we simulate reusable work types by maintaining a local dictionary 
of options and assigning them when creating Appointment records.
"""

WORK_TYPES = {
    "newpatient": {"system": "http://example.org/work-types", "code": "WT123", "display": "New Patient Consultation"},
    "followup": {"system": "http://example.org/work-types", "code": "WT456", "display": "Follow-up Visit"},
    "annual": {"system": "http://example.org/work-types", "code": "WT789", "display": "Annual Checkup"},
    "consult": {"system": "http://example.org/work-types", "code": "WT012", "display": "Consultation"},
    "nurse": {"system": "http://example.org/work-types", "code": "WT034", "display": "Nurse Visit"}
}

In [ ]:
# Appointments:
def fetch_appointments_for_practitioner(practitioner_id: str, base_url: str) -> dict:
    """
    Fetch Appointment records linked to a Practitioner (handles pagination).

    Args:
        practitioner_id (str): FHIR Practitioner resource ID.
        base_url (str): Base URL of the FHIR server.

    Returns:
        dict: JSON response with Appointment resources.
    """
    url = f"{base_url}/Appointment"
    params = {"actor": f"Practitioner/{practitioner_id}"}
    entries = fetch_all_resources(url, params)
    return {"entry": entries}


def appointments_to_df(response: dict) -> pd.DataFrame:
    """
    Normalize FHIR Appointment resources into a DataFrame with explicit actor IDs.

    Args:
        response (dict): JSON bundle with Appointment resources.

    Returns:
        pd.DataFrame: Flattened appointment records.
    """
    appointments = []
    for entry in response.get("entry", []):
        res = entry["resource"]

        appointment_id = res.get("id")
        status = res.get("status")
        start = res.get("start")
        end = res.get("end")

        # Participants
        practitioner_id = None
        patient_id = None
        location_id = None

        # Extract actors
        for p in res.get("participant", []):
            ref = p.get("actor", {}).get("reference", "")
            if ref.startswith("Practitioner/"):
                practitioner_id = ref.split("/")[-1]
            elif ref.startswith("Patient/"):
                patient_id = ref.split("/")[-1]
            elif ref.startswith("Location/"):
                location_id = ref.split("/")[-1]
                
        # Work Types
        work_type_code = None
        service_types = res.get("serviceType", [])
        if service_types and service_types[0].get("coding"):
            work_type_code = service_types[0]["coding"][0].get("code")
        
        appointments.append({
            "appointment_id": appointment_id,
            "patient_id": patient_id,
            "practitioner_id": practitioner_id,
            "location_id": location_id,
            "work_type_code": work_type_code,
            "status": status,
            "start": start,
            "end": end
        })

    return pd.DataFrame(appointments)

In [ ]:
all_appointments = []
for pid in providers_df["practitioner_id"]:
    response = fetch_appointments_for_practitioner(pid, BASE_URL)
    df = appointments_to_df(response)
    df["practitioner_id"] = pid  # keep track of provider
    all_appointments.append(df)

appointments_df = pd.concat(all_appointments, ignore_index=True)

In [ ]:
appointments_df = appointments_df.rename(columns=({
    'appointment_id': 'EHR_Appointment_Id__c',
    'status': 'Status'
}))
appointments_df['Status'] = appointments_df['Status'].replace({
    'booked': 'Scheduled',
    'cancelled': 'Canceled',
    'fulfilled': 'Completed'
})

In [ ]:
appointments_df['start'] = appointments_df['start'].str.replace('Z', '', regex=False).str.replace('00Z', '', regex=False)
appointments_df['end'] = appointments_df['end'].str.replace('Z', '', regex=False).str.replace('00Z', '', regex=False)

appointments_df['start'] = pd.to_datetime(appointments_df['start'], errors='coerce')
appointments_df['end'] = pd.to_datetime(appointments_df['end'], errors='coerce')

appointments_df['start'] = appointments_df['start'].dt.tz_localize(None)
appointments_df['end'] = appointments_df['end'].dt.tz_localize(None)

appointments_df['start'] = appointments_df['start'].dt.tz_localize(ZoneInfo('America/New_York'))
appointments_df['end'] = appointments_df['end'].dt.tz_localize(ZoneInfo('America/New_York'))

appointments_df['start_utc'] = appointments_df['start'].dt.tz_convert('UTC')
appointments_df['end_utc'] = appointments_df['end'].dt.tz_convert('UTC')

appointments_df['SchedStartTime'] = appointments_df['start_utc'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
appointments_df['SchedEndTime'] = appointments_df['end_utc'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')


### Salesforce Mapping

In [ ]:
# Salesforce Connection
sf = Salesforce(
    username=os.getenv('SF_USERNAME'),
    password=os.getenv('SF_PASSWORD'),
    security_token=os.getenv('SF_SECURITY_TOKEN'))

In [ ]:
# Dictionary to hold all the necessary ID maps
sf_id_maps = {}

account_query = """
    SELECT Id, EHR_Patient_Id__c
    FROM Account
    WHERE EHR_Patient_Id__c != null
"""
accounts_data = sf.query_all(account_query)['records']
sf_id_maps['patient_to_account'] = {rec['EHR_Patient_Id__c']: rec['Id'] for rec in accounts_data}

resource_query = """
    SELECT Id, EHR_Resource_Id__c, Name
    FROM ServiceResource
    WHERE EHR_Resource_Id__c != NULL AND IsActive = True
"""
resources_data = sf.query_all(resource_query)['records']
sf_id_maps['practitioner_to_resource'] = {rec['EHR_Resource_Id__c']: rec['Id'] for rec in resources_data}
sf_id_maps['resource_name_to_id'] = {rec['Name']: rec['Id'] for rec in resources_data}

contact_query = """
    SELECT Id, Name
    FROM Contact
"""
contact_data = sf.query_all(contact_query)['records']
name_to_contact_id = {rec['Name']: rec['Id'] for rec in contact_data}

sf_id_maps['practitioner_details'] = {}

for rec in resources_data:
    resource_name = rec['Name']
    ehr_id = rec['EHR_Resource_Id__c']
    resource_id = rec['Id']
    contact_id = name_to_contact_id.get(resource_name)
    if contact_id:
        sf_id_maps['practitioner_details'][ehr_id] = {
            'Service_Resource__c': resource_id,
            'ContactId': contact_id
        }
    else:
        print(f"⚠️ Warning: No matching Contact found for ServiceResource Name: {resource_name}")

territory_query = """
    SELECT Id, EHR_Location_Id__c
    FROM ServiceTerritory
    WHERE EHR_Location_Id__c != NULL AND IsActive = True
"""
territories_data = sf.query_all(territory_query)['records']
sf_id_maps['location_to_territory'] = {rec['EHR_Location_Id__c']: rec['Id'] for rec in territories_data}

worktype_query = """
    SELECT Id, EHR_Work_Type_Id__c
    FROM WorkType
    WHERE EHR_Work_Type_Id__c != NULL
"""
worktypes_data = sf.query_all(worktype_query)['records']
sf_id_maps['worktype_code_to_id'] = {rec['EHR_Work_Type_Id__c']: rec['Id'] for rec in worktypes_data}

print("✅ All ID maps created.")

In [ ]:
def prepare_service_appointments(appointments_df: pd.DataFrame, sf_maps: dict) -> pd.DataFrame:
    """
    Maps EHR IDs to Salesforce IDs and prepares the DataFrame for ServiceAppointment insertion.
    """
    sa_df = appointments_df.copy()

    # ParentId (Patient): Maps patient_id (EHR) to Account Id (SF)
    sa_df['ParentRecordId'] = sa_df['patient_id'].map(sf_maps['patient_to_account'])
    
    # Resource Details Retrieval (Single map lookup based on EHR ID)
    sa_df['resource_details'] = sa_df['practitioner_id'].map(sf_maps['practitioner_details'])
    
    # Extract Service_Resource__c
    sa_df['Service_Resource__c'] = sa_df['resource_details'].apply(
        lambda x: x['Service_Resource__c'] if isinstance(x, dict) else None
    )
    
    # Extract ContactId
    sa_df['ContactId'] = sa_df['resource_details'].apply(
        lambda x: x['ContactId'] if isinstance(x, dict) else None
    )
    
    # ServiceTerritoryId, WorkTypeId (Standard mappings)
    sa_df['ServiceTerritoryId'] = sa_df['location_id'].map(sf_maps['location_to_territory'])
    sa_df['WorkTypeId'] = sa_df['work_type_code'].map(sf_maps['worktype_code_to_id'])

    # Service Resource and Contact Mappings
    required_cols = ['ParentRecordId', 'Service_Resource__c', 'ContactId', 'WorkTypeId']
    sa_df_ready = sa_df.dropna(subset=required_cols).copy()
    
    skipped_count = len(sa_df) - len(sa_df_ready)
    if skipped_count > 0:
        print(f"⚠️ Skipped {skipped_count} Service Appointments due to missing mappings (Patient, Resource, Contact, or Work Type).")
    
    final_cols = [
        'EHR_Appointment_Id__c', 'ParentRecordId', 'Service_Resource__c', 'ContactId', 'ServiceTerritoryId', 'WorkTypeId',
        'SchedStartTime', 'SchedEndTime', 'Status'
    ]
    
    return sa_df_ready[final_cols]

In [ ]:
sa_to_load_df = prepare_service_appointments(appointments_df=appointments_df, sf_maps=sf_id_maps)